### 一、导入包

In [130]:
import sys
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

### 二、导入数据

In [131]:
#导入数据（前面‘’为数据存放路径，后面big5对字符串进行编码转换）
data = pd.read_csv('./data/train.csv',encoding='big5')

In [132]:
data

,Date,stations,observation,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2014/12/20,豐原,THC,1.8,1.8,1.8,1.8,1.8,1.7,1.7,...,1.8,1.8,2,2.1,2,1.9,1.9,1.9,2,2
4316,2014/12/20,豐原,WD_HR,46,13,61,44,55,68,66,...,59,308,327,21,100,109,108,114,108,109
4317,2014/12/20,豐原,WIND_DIREC,36,55,72,327,74,52,59,...,18,311,52,54,121,97,107,118,100,105
4318,2014/12/20,豐原,WIND_SPEED,1.9,2.4,1.9,2.8,2.3,1.9,2.1,...,2.3,2.6,1.3,1,1.5,1,1.7,1.5,2,2


### 三、处理数据

##### 处理空值数据并分割

In [133]:
#分割出前3列，从第4列开始将数据存到data
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Anaconda3\lib\site-packages\pandas\core\frame.py:2986: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [134]:
raw_data.shape

(4320, 24)

##### 将数据调整为12 * 18 * 480

In [135]:
#对data进行调整，将4320*24重组为12*18*480
month_data = {}
for month in range(12):
    sample = np.empty([18,480])
    for day in range(20):
        sample[:,day * 24 : ( day + 1 ) * 24] = raw_data [ 18 * ( 20 * month + day ) : 18 * ( 20 * month + day + 1 ),: ]
    month_data[month] = sample

In [136]:
# 12*18*480
month_data

{0: array([[14.  , 14.  , 14.  , ..., 14.  , 13.  , 13.  ],
        [ 1.8 ,  1.8 ,  1.8 , ...,  1.8 ,  1.8 ,  1.8 ],
        [ 0.51,  0.41,  0.39, ...,  0.34,  0.41,  0.43],
        ...,
        [35.  , 79.  ,  2.4 , ..., 48.  , 63.  , 53.  ],
        [ 1.4 ,  1.8 ,  1.  , ...,  1.1 ,  1.9 ,  1.9 ],
        [ 0.5 ,  0.9 ,  0.6 , ...,  1.2 ,  1.2 ,  1.3 ]]),
 1: array([[ 15.  ,  14.  ,  14.  , ...,   8.4 ,   8.  ,   7.6 ],
        [  1.8 ,   1.8 ,   1.7 , ...,   1.7 ,   1.7 ,   1.7 ],
        [  0.27,   0.26,   0.25, ...,   0.36,   0.35,   0.32],
        ...,
        [113.  , 109.  , 104.  , ...,  72.  ,  65.  ,  69.  ],
        [  2.3 ,   2.2 ,   2.6 , ...,   1.9 ,   2.9 ,   1.5 ],
        [  2.5 ,   2.2 ,   2.2 , ...,   0.9 ,   1.6 ,   1.1 ]]),
 2: array([[ 18.  ,  18.  ,  18.  , ...,  14.  ,  13.  ,  13.  ],
        [  1.8 ,   1.8 ,   1.8 , ...,   1.8 ,   1.8 ,   1.8 ],
        [  0.39,   0.36,   0.4 , ...,   0.42,   0.47,   0.49],
        ...,
        [103.  , 128.  , 115.  , ...,  

##### 将数据调整为[12 * 471,18 * 9]，行代表数据个数，列代表特征


In [137]:
x = np.empty([12*471,18*9],dtype = float)
y = np.empty([12*471,1],dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour>14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1,-1)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9,day * 24 + hour + 9]

In [138]:
x.shape

(5652, 162)

In [139]:
y.shape

(5652, 1)

##### 归一化处理

In [140]:
#归一化
mean_x = np.mean(x,axis = 0)
std_x = np.std(x,axis = 0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

##### 划分训练集和验证集

In [141]:
#将训练集分成训练-验证集，用来最后检验我们的模型
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]

### 四、训练模型

##### 对数据增维，把b和w放在一起，数据增加一列1

In [142]:
#因为存在偏差bias，所以dim+1
dim = 18 * 9 + 1
# w维度为163*1
w = np.zeros([dim,1])
# x_train_set维度为 4521*163
# concatenate为拼接矩阵函数，axis=0拼接行，axis=1拼接列,第一列加一列1,与参数b对应
x_train_set= np.concatenate((np.ones([len(x_train_set),1]),x_train_set),axis = 1).astype(float)

##### 初始化参数

In [143]:
#设置学习率
learning_rate = 10
#设置迭代数
iter_time = 30000
#RMSprop参数初始化
adagrad = np.zeros([dim,1])
eps = 0.0000000001
#beta = 0.9
lambda2 = 20

##### Adgrade梯度下降

In [144]:
#迭代
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x_train_set,w)-y_train_set,2))/len(x_train_set))
    if(t%100 == 0):
        print("迭代的次数：%i ， 损失值：%f"%(t,loss))
    #计算梯度值
    # gradient = 2*np.dot(x_train_set.transpose(),np.dot(x_train_set,w)-y_train_set)

    gradient = (np.dot(x_train_set.transpose(),np.dot(x_train_set,w)-y_train_set))/(loss*len(x_train_set))
    adagrad += (gradient ** 2)
    #更新参数w
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)  
    # w = w - learning_rate * gradient / np.sqrt(adagrad)    

迭代的次数：0 ， 损失值：27.239592
迭代的次数：100 ， 损失值：34.254203
迭代的次数：200 ， 损失值：23.456061
迭代的次数：300 ， 损失值：19.822073
迭代的次数：400 ， 损失值：16.896571
迭代的次数：500 ， 损失值：15.140156
迭代的次数：600 ， 损失值：14.207356
迭代的次数：700 ， 损失值：13.100742
迭代的次数：800 ， 损失值：12.279474
迭代的次数：900 ， 损失值：11.625636
迭代的次数：1000 ， 损失值：11.086533
迭代的次数：1100 ， 损失值：10.591193
迭代的次数：1200 ， 损失值：10.366615
迭代的次数：1300 ， 损失值：9.969749
迭代的次数：1400 ， 损失值：9.661329
迭代的次数：1500 ， 损失值：9.392065
迭代的次数：1600 ， 损失值：9.110842
迭代的次数：1700 ， 损失值：8.979510
迭代的次数：1800 ， 损失值：8.785610
迭代的次数：1900 ， 损失值：8.598740
迭代的次数：2000 ， 损失值：8.429441
迭代的次数：2100 ， 损失值：8.275065
迭代的次数：2200 ， 损失值：8.133990
迭代的次数：2300 ， 损失值：8.002266
迭代的次数：2400 ， 损失值：7.945385
迭代的次数：2500 ， 损失值：7.804851
迭代的次数：2600 ， 损失值：7.706406
迭代的次数：2700 ， 损失值：7.610114
迭代的次数：2800 ， 损失值：7.516768
迭代的次数：2900 ， 损失值：7.430999
迭代的次数：3000 ， 损失值：7.349425
迭代的次数：3100 ， 损失值：7.276118
迭代的次数：3200 ， 损失值：7.210737
迭代的次数：3300 ， 损失值：7.122030
迭代的次数：3400 ， 损失值：7.115478
迭代的次数：3500 ， 损失值：7.046100
迭代的次数：3600 ， 损失值：6.985730
迭代的次数：3700 ， 损失值：6.931262
迭代的次数：3800 

##### 保存参数

In [145]:
#保存参数w
np.save('weight.npy',w)

##### 对测试集预处理

In [146]:
# 对测试集预处理
testdata = pd.read_csv('./data/test.csv',header = None ,encoding = 'big5')
test_data = testdata.iloc[:,2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()

test_x = np.empty([240,18*9],dtype = float)
for i in range(240):
    test_x[i,:] = test_data[18*i:18*(i+1),:].reshape(1,-1)

# 均一化
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
            
# 添加一列1
test_x = np.concatenate((np.ones([240,1]),test_x),axis = 1).astype(float)

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [147]:
test_x

array([[ 1.        , -1.19914238, -1.35883937, ...,  0.73508789,
         0.73411671,  0.54674825],
       [ 1.        , -1.67647517, -1.67694799, ...,  0.36022341,
         0.82780412, -0.20328266],
       [ 1.        , -2.18563014, -1.67694799, ...,  0.82880401,
         0.0783049 , -0.29703653],
       ...,
       [ 1.        , -1.67647517, -1.8360023 , ...,  0.07907505,
        -0.10906991, -1.04706744],
       [ 1.        , -1.35825331, -1.51789368, ..., -0.10835719,
         0.35936711,  0.07797893],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131]])

##### 在验证集上验证

In [148]:
#在验证集上进行验证
w = np.load('weight.npy')
x_validation= np.concatenate((np.ones([len(x_validation),1]),x_validation),axis = 1).astype(float)
for m in range(len(x_validation)):
    Loss = np.sqrt(np.sum(np.power(np.dot(x_validation,w)-y_validation,2))/len(x_validation))
print ("the Loss on val data is %f" % (Loss))
#预测
ans_y = np.dot(test_x, w)
print('预测PM2.5值')
print(ans_y)

the Loss on val data is 5.664584
预测PM2.5值
[[ 24.89783688]
 [ 64.19399589]
 [ 19.0187991 ]
 [ 29.35821884]
 [  9.71530416]
 [ 33.08488639]
 [ 40.80956735]
 [ 18.31685814]
 [ 54.18480397]
 [ 32.14411323]
 [ 38.16911005]
 [ 59.00940273]
 [ 35.70073879]
 [ 41.11644963]
 [ 27.00165007]
 [ 13.09309667]
 [ 51.61950008]
 [ 25.38365827]
 [ 18.63010289]
 [ 20.26992513]
 [ 16.5821156 ]
 [ 32.1056491 ]
 [ 39.9413303 ]
 [ 12.3414687 ]
 [ 61.57176888]
 [ 46.02644035]
 [ 28.66440458]
 [  6.06500206]
 [ 28.57108931]
 [ 37.95868015]
 [ 17.73929008]
 [ 11.60978218]
 [ 17.08087737]
 [  7.01979218]
 [ 15.17907419]
 [ 40.89698742]
 [ 22.16208087]
 [ 28.33917865]
 [  5.55338158]
 [ 60.01537246]
 [ 47.4217705 ]
 [ 21.03019675]
 [ 52.6999865 ]
 [ 19.97368907]
 [ 44.04673639]
 [ 48.94374417]
 [ 57.84052904]
 [ 41.91104148]
 [ 55.54008244]
 [ 39.49082613]
 [ 30.98217475]
 [ 24.07599495]
 [ 39.16949935]
 [ 23.36225333]
 [ 37.18460961]
 [ 38.70186031]
 [ 25.72009578]
 [ 22.32785417]
 [ 35.55432707]
 [ 32.155486  

In [149]:
# 保存为csv文件，并提交到kaggle：https://www.kaggle.com/c/ml2020spring-hw1/submissions
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 24.897836876803368]
['id_1', 64.19399588842123]
['id_2', 19.01879910135813]
['id_3', 29.358218835624214]
['id_4', 9.715304156753906]
['id_5', 33.08488638570235]
['id_6', 40.80956734715575]
['id_7', 18.316858138388895]
['id_8', 54.18480396660377]
['id_9', 32.144113234049115]
['id_10', 38.16911005222184]
['id_11', 59.00940273142856]
['id_12', 35.70073878689048]
['id_13', 41.11644962841391]
['id_14', 27.00165006662118]
['id_15', 13.093096674394268]
['id_16', 51.6195000758691]
['id_17', 25.3836582747843]
['id_18', 18.630102886061216]
['id_19', 20.269925131793844]
['id_20', 16.58211559917281]
['id_21', 32.10564910073492]
['id_22', 39.94133029795437]
['id_23', 12.341468704080231]
['id_24', 61.571768877166406]
['id_25', 46.02644034810406]
['id_26', 28.66440457834399]
['id_27', 6.065002055202925]
['id_28', 28.57108931071813]
['id_29', 37.95868014991593]
['id_30', 17.739290078833214]
['id_31', 11.609782180686565]
['id_32', 17.08087736985567]
['id_33', 7.019792184685649]